In [1]:
import numpy as np
import pylab
import plotly.graph_objects as go
import time

import pylab
import matplotlib
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit.opflow import X, Z, Y, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.circuit.library import TwoLocal, QAOAAnsatz

from qiskit.algorithms.optimizers import ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, \
                                            NFT, POWELL, SLSQP, SPSA, TNC


#Imports Functions for Building Hamiltonains
%run Utils.ipynb
np.set_printoptions(formatter={'float': '{:0.3f}'.format})

# Parameters

In [9]:
#use even number of qubits for Schwinger
N = 12 #qubits
g = 1 #coupling
m = 1 #bare mass
a = .25 #lattice spacing
theta = 0 #topological term

# Coefficient Matrix

In [5]:
def render_landscape(N_range, quality):
    renders = {}
    for N in range(2, N_range + 1,2):
        print(f"Rendering {N} Qubit Landscape...")
        target_coupling, mixer_coupling = makeSchwinger(N, (g**2)*a / 2, m / 2, 1 / (4 * a))
        mixer_hamiltonian = mixer_coupling.getHamiltonian()
        target_hamiltonian = target_coupling.getHamiltonian()

        ansatz = QAOAAnsatz(cost_operator= target_hamiltonian, 
                            reps=1, 
                            initial_state=None, 
                            mixer_operator= mixer_hamiltonian, 
                            name='QAOA')

        complete_hamiltonian = mixer_hamiltonian + target_hamiltonian

        min_coeff = min(abs(target_hamiltonian.coeffs))

        vqe = VQE(ansatz, quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))
        
        brr, grr = 1, 1

        gamma_range = np.linspace(0, (2/min_coeff)*np.pi * grr, int(quality * 1/(min_coeff)) + 1)
        beta_range = np.linspace(0, (2/min_coeff)*np.pi * brr, int(quality * 1/(min_coeff)) + 1)
        
        get_energy = vqe.get_energy_evaluation(complete_hamiltonian)
        calcEnergy = lambda x,y: get_energy([x,y])
        
        start = time.time()

        gamma, beta, energy = renderLandscape(gamma_range, beta_range, calcEnergy)
        renders[N] = (gamma, beta, energy)
        end = time.time()
        print(f"Rendered In: {end - start : .2f}s")
    print("Done!    ")   
    return renders

renders = render_landscape(N, 16)

Rendering 2 Qubit Landscape...
Rendered In:  19.71s
Done!    


In [7]:
xrr = (0, 1) #beta
yrr = (0, 1) #gamma
size = renders[list(renders)[0]][0].shape
xrr = (int(np.ceil(xrr[0] * size[0])),int(np.ceil(xrr[1] * size[0])))
yrr = (int(np.ceil(yrr[0] * size[1])), int(np.ceil(yrr[1] * size[1])))
print(xrr,yrr)

in_range = lambda coords: coords[xrr[0]:xrr[1],yrr[0]:yrr[1]]

fig = go.Figure(data=[go.Surface(x=in_range(renders[r][0]), y=in_range(renders[r][1]), z=in_range(renders[r][2]) + 3*r, opacity=1, showscale=False, colorscale='Viridis') for r in renders.keys()])

fig.update_layout(title='Energy Landscape',
                   autosize=False,
                  width=1000, height=1000,
                 scene = dict(
                    xaxis_title='Gamma',
                    yaxis_title='Beta',
                    zaxis_title='Energy'))
fig.show()

(0, 65) (0, 65)


In [37]:
n = 12

fig = go.Figure(data =
    go.Contour(x = renders[n][0][0], y = renders[n][1][:,0], z=renders[n][2], colorscale='Viridis', contours_coloring='heatmap', line_width = 0
    ))
fig.update_traces(colorbar_title = 'Energy')

fig.update_layout(title='Energy Height Map', xaxis_title='Gamma', yaxis_title='Beta', autosize=False,
                  width=900, height=900,
                 )
fig.show()